In [ ]:
import math
from glob import glob
import os

import cv2
import numpy as np
from PIL import Image
from pdf2image import convert_from_path
from imutils import resize
from sklearn.metrics import euclidean_distances
from tqdm import tqdm
from scoring import *
from border_and_title import *

files = [
    "../../data/original/Good/300-014820.pdf",
    "../../data/original/Good/300-014823.pdf",
    "../../data/original/Good/300-014821.pdf",
    "../../data/original/Good/300-014822.pdf"
]
# files = [
#     "../../data/original/Ugly/000.001.360.pdf",
#     "../../data/original/Ugly/000.000.537.pdf"
# ]

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0)
thickness = 2

def read_pdf(pdf_path):
    pages = convert_from_path(pdf_path, dpi=300)
    img = np.array(pages[0])
    img = im_resize(img)
    return img


In [ ]:
images = []
cn_images = None


for file in files:
    img = read_pdf(file)
    images.append(img)
    print(images[-1].shape)

    if cn_images is None:
        cn_images = img
    else:
        cn_images = np.hstack((cn_images, img))


Image.fromarray(cn_images)

In [ ]:
from skimage.metrics import structural_similarity
import cv2

def difference(image1, image2):
    if len(image1.shape) == 3:
        image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)

    if len(image2.shape) == 3:
        image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)

    (score, diff) = structural_similarity(image1, image2, full=True)

    diff = (diff * 255).astype("uint8")
    print("Image Similarity: {:.4f}%".format(score * 100))



    return diff, np.hstack((image1, image2, diff))

diff, con = difference(images[0], images[1])
Image.fromarray(con)

In [ ]:
diff, con = difference(images[1], images[2])
Image.fromarray(con)

In [ ]:
diff, con = difference(images[2], images[3])
Image.fromarray(con)

In [ ]:
masks = []
no_texts = []

for img in images:
    title_contours, line_90, data = detect_probable_title_sections(img, return_states=True)
    ocr_result = data["ocr_result"]
    mask = data["mask"]
    masks.append(mask)

    im_h, im_w, _ = img.shape
    words = process_text(ocr_result, im_h, im_w)
    img_comp = img.copy()

    for idx, row in words.iterrows():
        bx1, by1, bx2, by2 = row[["x1", "y1", "x2", "y2"]].astype(int)
        cv2.rectangle(img_comp, (bx1, by1), (bx2, by2), (255, 255, 255), cv2.FILLED)

    no_texts.append(img_comp)

In [ ]:
diff, con = difference(masks[0], masks[1])
Image.fromarray(con)

In [ ]:
diff, con = difference(masks[1], masks[2])
Image.fromarray(con)

In [ ]:
diff, con = difference(masks[2], masks[3])
Image.fromarray(con)

In [ ]:
diff, con = difference(no_texts[0], no_texts[1])
Image.fromarray(con)

In [ ]:
diff, con = difference(no_texts[1], no_texts[2])
Image.fromarray(con)

In [ ]:
diff, con = difference(no_texts[2], no_texts[3])
Image.fromarray(con)

In [ ]:
Image.fromarray(diff)